In [1]:
from nltk.util import ngrams
from collections import defaultdict
import re
import numpy as np
from itertools import product
from tqdm import tqdm

## Часть 1.

Реализуйте базовый частотный метод по Шерлоку Холмсу:
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

Решение:

Напишем функции, осуществляющие подсчет частот в файле (для оценки теоретических частот языка) и в фрагменте текста:

In [2]:
def get_ngrams_freqs(filepath, ngram_size=1):
    tuple_freqs = defaultdict(int)
    ngram_count = 0
    buf = ""
    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            clear_line = buf + " ".join(re.findall(r'[А-я]+', line)).lower()
            for ng in ngrams(clear_line, n=ngram_size):
                tuple_freqs[ng] += 1
            clear_line += " "
            ngram_count += len(clear_line) - ngram_size
            buf = clear_line[-ngram_size:]
    output = defaultdict(int)
    for sym, freq in tuple_freqs.items():
        output["".join(list(sym))] = freq / ngram_count
    return output


def get_sample_freqs(text, ngram_size=1):
    tuple_freqs = defaultdict(int)
    clear_text = " ".join(re.findall(r'[А-я]+', text)).lower()   
    for ng in ngrams(clear_text, n=ngram_size):
            tuple_freqs[ng] += 1
    ngram_count = len(clear_text) - ngram_size
    output = {}
    for sym, freq in tuple_freqs.items():
        output["".join(list(sym))] = freq / ngram_count
    return output

etalon_freqs = get_ngrams_freqs('AnnaKarenina.txt')
alphabet = ' абвгдежзийклмнопрстуфхцчшщъыьэюя'

Также реализуем функции, осуществляющие шифр простой замены и дешифрование сообщения. Для оценки качества также потребуется метрика качества. Будем пользоваться точностью - долей верно угаданных символов шифра замены:

In [3]:
def simple_substitution_cipher(
    plaintext,
    alphabet=' абвгдежзийклмнопрстуфхцчшщъыьэюя',
    seed=42):
    np.random.seed(seed)
    permute = np.random.permutation(list(alphabet))
    substitution = {
        k: v for k, v in zip(alphabet, permute)
    }
    return ("".join([substitution[sym] for sym in plaintext]), substitution)


def decrypt_substitution_cipher(cipher_text, decrypt_substitution):
    return "".join(decrypt_substitution[sym] for sym in cipher_text)


def decryption_quality(pred_sub, cipher_sub):
    success_count = 0
    for cipher_sym, plain_sym in pred_sub.items():
        if cipher_sub[plain_sym] == cipher_sym:
            # print(plain_sym, "->", cipher_sym)
            success_count += 1
    return success_count / len(pred_sub)

Выберем фрагмент текста из произведения и зашифруем его случайной перестановкой символов:

In [4]:
sample_text = 'потребность жизни увеличенная выздоровлением была так сильна и условия жизни были так новы и приятны что анна чувствовала себя непростительно счастливою чем больше она узнавала вронского тем больше она любила его она любила его за его самого и за его любовь к ней полное обладание им было ей постоянно радостно близость его ей всегда была приятна все черты его характера который она узнавала больше и больше были для нее невыразимо милы'

In [5]:
cipher_text, cipher_substitution = simple_substitution_cipher(sample_text)
cipher_text

'ачвбтщкчьвжюф лк юъртд стккоыюряличбчрдтк тмющядоювопюь джкою юъьдчр ыюф лк ющяд ювопюкчряю юаб ывкяюсвчюоккоюсърьврчродоюьтщыюктабчьв втджкчюьсоьвд рчнюстмющчджштючкоюълкородоюрбчкьпчзчювтмющчджштючкоюднщ доютзчючкоюднщ доютзчюлоютзчюьомчзчю юлоютзчюднщчржюпюктгюачдкчтючщдоиок тю мющядчютгюачьвчыккчюбоичьвкчющд лчьвжютзчютгюрьтзиоющядоюаб ывкоюрьтюстбвяютзчюэобопвтбоюпчвчбягючкоюълкородоющчджштю ющчджштющяд юидыюкттюктрябол мчюм дя'

Для дешифровки с помощью n-грамм символов будем использовать следующий класс, который осуществляет жадный алгоритм сопоставления частот открытого и шифртекстов (для букв - сопоставляет символы по частоте, для n-грам - ищет наиболее вероятную n-грамму открытого текста, которая может быть сопоставлена рассматриваемой в открытом тексте):

In [6]:
class NgramDecryptor:
    def __init__(
        self,
        ngram_size=1,
        etalon_freqs=etalon_freqs):
        self.ngram_size = ngram_size
        self.most_popular_origin = [sym for sym, _ in sorted(etalon_freqs.items(), key=lambda x: x[1], reverse=True)]
        self.most_popular_cipher = None
        self.cipher_syms = None
        self.decrypt_substitution = {}
    
    def fit(self, cipher_text):
        cipher_text_freqs = get_sample_freqs(cipher_text, ngram_size=self.ngram_size)
        self.most_popular_cipher = [sym for sym, freq in sorted(cipher_text_freqs.items(), key=lambda x: x[1], reverse=True)]
        self.cipher_syms = set(cipher_text)
        for i in range(len(self.most_popular_cipher)):
            j = 0
            try:
                while not self.__could_assign(
                    self.most_popular_cipher[i],
                    self.most_popular_origin[j],):
                    j += 1
                for ng_c, ng_e in zip(
                    self.most_popular_cipher[i],
                    self.most_popular_origin[j]):
                    self.decrypt_substitution[ng_c] = ng_e
                    if ng_c in self.cipher_syms:
                        self.cipher_syms.remove(ng_c)
                if not len(self.cipher_syms):
                    break
            except IndexError: # haven't found good mapping, better skip it
                pass

    def __could_assign(self, cipher_ngram, etalon_ngram):
        # ngrams syms was added before, no reason to do this step
        if not self.cipher_syms.intersection(set(cipher_ngram)):
            return False
        
        loc_dict = {} # keep it to avoid cases like 'aba' -> 'gbx',
                      # where `a` has to be 'a': 'g' and 'x'.

        for sym_c, sym_e in zip(cipher_ngram, etalon_ngram):
            if sym_c in self.decrypt_substitution.keys():
                if self.decrypt_substitution[sym_c] != sym_e:
                    return False
                else:
                    pass # mapped in global dict
            else:
                if sym_e in self.decrypt_substitution.values():
                    return False
                else:
                    if sym_e not in loc_dict.values():
                        loc_dict[sym_c] = sym_e
                    else:
                        if sym_c not in loc_dict or loc_dict[sym_c] != sym_e:
                            return False
        return True

    def predict(self, cipher_text):
        return decrypt_substitution_cipher(cipher_text, self.decrypt_substitution)

Проверим качество дешифрования используя частоты символов:

In [7]:
dec = NgramDecryptor(ngram_size=1)
dec.fit(cipher_text)
dec.predict(cipher_text)

'ыоскалнорсм хтунт чваитганнея вдузоковиантаь лдие себ ртимне т чриовтя хтунт лдит себ новд т ыктяснд гсо енне гчврсвовеие раля наыкорстсаимно ргерситвож гаь лоимйа оне чуневеие вконрбопо саь лоимйа оне ижлтие апо оне ижлтие апо уе апо реьопо т уе апо ижловм б наш ыоиноа олиезента ть лдио аш ыорсоянно кезорсно литуорсм апо аш врапзе лдие ыктясне вра гаксд апо юекебсаке босокдш оне чуневеие лоимйа т лоимйа лдит зия наа навдкеутьо ьтид'

In [8]:
decryption_quality(dec.decrypt_substitution, cipher_substitution)

0.17857142857142858

4 символа были сопоставлены верно, однако текст все равно остается нечитаемым.

## Часть 2.

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.


Сделаем аналогичную процедуру с биграммами текста:

In [9]:
etalon_bigram_freqs=get_ngrams_freqs('AnnaKarenina.txt', ngram_size=2)
dec = NgramDecryptor(ngram_size=2,  etalon_freqs=etalon_bigram_freqs)
dec.fit(cipher_text)
dec.predict(cipher_text)

'пекхинтескл зачта вмигадиттоя мычжехемгитаиь ныго коб саглто а всгемая зачта ныга коб темы а пхаякты дке отто двмскмемого синя типхескакиглте сдоскгамеш диь неглуи ето вчтомого мхетсбере киь неглуи ето гшнаго ире ето гшнаго ире чо ире соьере а чо ире гшнемл б тий пегтеи енгожотаи аь ныге ий пескеятте хожескте нгаческл ире ий мсиржо ныго пхаякто мси дихкы ире эохобкихо бекехый ето вчтомого неглуи а неглуи ныга жгя тии тимыхочаье ьагы'

In [10]:
decryption_quality(dec.decrypt_substitution, cipher_substitution)

0.21428571428571427

Получилось несколько успешнее, однако это мало о чем говорит, ведь текст по-прежнему нечитаем.

## Часть 3.

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


В данном задании будем рассматривать МСМС-сэмплинг как способ нахождения максимума плотности апостериорной вероятности подстановки при условии наблюдаемого текста.
Будем осуществлять перестановку случайно взятой пары символов, основываясь на изменении вероятностей наблюдения дешифрованных (с помощью текущей и модернизированной подстановок) текстов. Эти вероятности будем вычислять на основе статистике биграм открытых текстов. Из технических соображений будет оперировать логарифмами правдоподобий текстов. В силу симметрии функционала замены случайной пары символов, МСМС шаг выглядит следующим образом:

Пусть на шаге $t$ мы находимся в состоянии $d_t$ (то есть имеем некоторую перестановку). Здесь также отметим, что в качестве начального приближения можно воспользоваться перестановкой, основанной на дешифровании с помощью частот знаков/биграмм.

1) Проводим случайную перестановку двух символов в $d_t$, получаем перестановку $d'$.

2) Расшифровываем шифртекст $text$ используя перестановки $d_t$ и $d'$ и вычисляем правдоподобия полученных открытых текстов.

3) Вычисляем $a = \frac{p(text|d')}{p(text|d_t)}$

4) Если $a >= 1$, тогда $d_{t+1} = d'$. Иначе, $d_{t+1} = d'$ с вероятностью $a$ или $d_{t+1}=d_t$ с вероятностью $1-a$ соответственно.

Из технических соображений, вычислим набор значений логарифмов вероятностей.
Кроме того, возможно потребуется дополнить перестановку до полной (в наблюдаемом шифртексте мощность алфавита может быть меньше, чем в открытом тексте).

In [11]:
eps = 1e-6
etalon_log_freqs = {}
for i, j in product(alphabet, repeat=2):
    etalon_log_freqs[i + j] = np.log(etalon_bigram_freqs[i + j] + eps)

In [12]:
def extend_substitution_to_full(sub, etalon_freqs):
    most_popular_origin = [sym for sym, freq in sorted(etalon_freqs.items(), key=lambda x: x[1], reverse=True)]
    for sym in most_popular_origin:
        if sym not in sub.keys():
            sub[sym] = sym
    return sub
    
decrypt_substitution = extend_substitution_to_full(dec.decrypt_substitution, etalon_freqs)

Реализуем функцию, осуществляющую МСМС-сэмплирование для указанной задачи.

In [13]:
def mcmc_sampling(
    sample_text,
    init_substitution,
    alphabet=' абвгдежзийклмнопрстуфхцчшщъыьэюя',
    etalon_log_freqs=etalon_log_freqs,
    n_steps=10000,
    ngram_size=2,
    seed=123,
):
    np.random.seed(seed)
    best_sub = None
    best_llh = -np.inf
    alphabet_symbols = list(alphabet)
    curr_sub = init_substitution.copy()
    curr_sub_llh = __text_loglikelihood(
        decrypt_substitution_cipher(sample_text, curr_sub),
        etalon_log_freqs, ngram_size=ngram_size)
    for _ in tqdm(range(n_steps)):
        # create MC step
        idx_first = np.random.choice(alphabet_symbols)
        idx_second = np.random.choice(alphabet_symbols)
        next_sub = curr_sub.copy()
        next_sub[idx_first], next_sub[idx_second] = curr_sub[idx_second], curr_sub[idx_first]
        # score to either reject or accept
        next_sub_decrypt_text = decrypt_substitution_cipher(sample_text, next_sub)
        next_sub_llh = __text_loglikelihood(next_sub_decrypt_text, etalon_log_freqs, ngram_size=ngram_size)
        if __mcmc_accept(curr_sub_llh, next_sub_llh):
            curr_sub = next_sub.copy()
            curr_sub_llh = next_sub_llh
            if curr_sub_llh > best_llh:
                best_sub = curr_sub.copy()
                best_llh = curr_sub_llh
                print('\nNew best llh: ', best_llh)
                print('Best decryption: ', next_sub_decrypt_text)
    return best_sub


def __mcmc_accept(curr_llh, next_llh):
    prob = min(1, np.exp(next_llh - curr_llh))
    if prob >= 1:
        return True
    else:
        return np.random.random() < prob


def __text_loglikelihood(text, etalon_log_freqs, ngram_size=2):
    llh = 0
    for i in range(len(text) - ngram_size + 1):
        llh += etalon_log_freqs[text[i:i + ngram_size]]
    return llh

Запустим и оценим результаты (здесь в качестве начального приближения используется перестановка, основанная на частотах встречаемости биграмм):

In [14]:
best_substitution = mcmc_sampling(cipher_text, decrypt_substitution)
print("\n\nBest decryption: \n", decrypt_substitution_cipher(cipher_text, best_substitution))
print("Accuracy: ", decryption_quality(best_substitution, cipher_substitution))

  7%|▋         | 746/10000 [00:00<00:02, 3628.22it/s]
New best llh:  -2795.138545292259
Best decryption:  пекхинтескл зачта вмигадиттоя мычжехемгитаиь ныго коб саглто а всгемая зачта ныга коб темы а пхаякты дке отто двмскмемого синя типхескакиглте сдоскгамеш диь неглуи ето вчтомого мхетсбере киь неглуи ето гшнаго ире ето гшнаго ире чо ире соьере а чо ире гшнемл б тий пегтеи енгожотаи аь ныге ий пескеятте хожескте нгаческл ире ий мсиржо ныго пхаякто мси дихкы ире эохобкихо бекехый ето вчтомого неглуи а неглуи ныга жгя тии тимыхочаье ьагы

New best llh:  -2789.3485154624204
Best decryption:  пекхинтескл зачта вмигаьиттоя мычжехемгитаид ныго коб саглто а всгемая зачта ныга коб темы а пхаякты ьке отто ьвмскмемого синя типхескакиглте сьоскгамеш ьид неглуи ето вчтомого мхетсбере кид неглуи ето гшнаго ире ето гшнаго ире чо ире содере а чо ире гшнемл б тий пегтеи енгожотаи ад ныге ий пескеятте хожескте нгаческл ире ий мсиржо ныго пхаякто мси ьихкы ире эохобкихо бекехый ето вчтомого неглуи а не

Получили очень высокую точность (выше, чем я ожидал) и практически идеально читаемый текст.

## Часть 4.

Расшифруйте сообщение:

In [15]:
test_cipher_text = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

Без потери общности, перейдем от имеющихся символов к алфавиту русского языка (чисто техническое соображение):

In [16]:
syms_to_ru = {}
for sym in test_cipher_text:
    if sym not in syms_to_ru.keys():
        syms_to_ru[sym] = alphabet[len(syms_to_ru)]

test_cipher_text = "".join([syms_to_ru[sym] for sym in test_cipher_text])
test_cipher_text

' абвгдегдвжвз гийклмбниеогвбвгпйрзвгийклмбниеогз сазгтгузйфйгаййхц ивчгсйзйкеогб фсйгпкйрвзмзнгасйк  гда фйгдегда гаж бмбвгпкмдвбнийгвгпйбтрвз глмсавлмбниеогхмббгшмгпйаб жи  гр зд кзй гшмжмив гсткамгщйзчгсйи рийгчгивр фйги гйх цмъ'

В качестве начального приближения возьмем перестановку, основанную на частотах встречаемости символов:

In [17]:
dec_test = NgramDecryptor(ngram_size=1)
dec_test.fit(test_cipher_text)
decrypt_substitution = extend_substitution_to_full(dec_test.decrypt_substitution, etalon_freqs)

In [18]:
best_substitution = mcmc_sampling(test_cipher_text, decrypt_substitution, n_steps=20000)
print("\nBest decryption: \n", decrypt_substitution_cipher(test_cipher_text, best_substitution))

  3%|▎         | 589/20000 [00:00<00:03, 5860.76it/s]
New best llh:  -1511.211634967785
Best decryption:  елна рд раяасе иовьтнуиды ана помса иовьтнуиды секлс б йсого лоочжеиаз косовды негко пвомастсу лковее рлего рд рле ляентна пвтрануио а понбмасе ьтклаьтнуиды чтнн шт полнеяиее месревсое штятиае кбвлт хосз коиемио з иамего ие очежтю

New best llh:  -1495.5580873788285
Best decryption:  елна рд раяасе иовьтнбиды ана помса иовьтнбиды секлс у йсого лоочжеиаз косовды негко пвомастсб лковее рлего рд рле ляентна пвтранбио а понумасе ьтклаьтнбиды чтнн шт полнеяиее месревсое штятиае кувлт хосз коиемио з иамего ие очежтю

New best llh:  -1487.0427906217942
Best decryption:  елна рд разасе иовьтнбиды ана помса иовьтнбиды секлс у йсого лоочжеиая косовды негко пвомастсб лковее рлего рд рле лзентна пвтранбио а понумасе ьтклаьтнбиды чтнн шт полнезиее месревсое штзтиае кувлт хося коиемио я иамего ие очежтю

New best llh:  -1480.035291691983
Best decryption:  елна рд разасе иовьтнбиды ана помса иовь

Текст практически нормальный, оставшиеся символы легко восстанавливаются при необходимости точного дешифрования.

## Часть 5.

Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

Для примера проведем аналогичные п.2, 3 вычисления, основываясь на триграммах текста.

In [19]:
etalon_trigram_freqs = get_ngrams_freqs('AnnaKarenina.txt', ngram_size=3)
eps = 1e-6
etalon_log_freqs = {}
for i, j, k in product(alphabet, repeat=3):
    etalon_log_freqs[i + j + k] = np.log(etalon_trigram_freqs[i + j + k] + eps)

In [20]:
decrypt_substitution = extend_substitution_to_full(dec.decrypt_substitution, etalon_freqs)
best_substitution = mcmc_sampling(cipher_text, decrypt_substitution, etalon_log_freqs=etalon_log_freqs, ngram_size=3, n_steps=15000)
print("\nBest decryption: \n", decrypt_substitution_cipher(cipher_text, best_substitution))
print("Accuracy: ", decryption_quality(best_substitution, cipher_substitution))

  2%|▏         | 331/15000 [00:00<00:04, 3293.64it/s]
New best llh:  -4330.609425096292
Best decryption:  пекхинтескл зачта вмигадиттоя мычжехемгитаиь ныго коб саглто а всгемая зачта ныга коб темы а пхаякты дке отто двмскмемого синя типхескакиглте сдоскгамеш диь неглуи ето вчтомого мхетсбере киь неглуи ето гшнаго ире ето гшнаго ире чо ире соьере а чо ире гшнемл б тий пегтеи енгожотаи аь ныге ий пескеятте хожескте нгаческл ире ий мсиржо ныго пхаякто мси дихкы ире эохобкихо бекехый ето вчтомого неглуи а неглуи ныга жгя тии тимыхочаье ьагы

New best llh:  -4318.7271317116
Best decryption:  пекхинтескл задта вмигачиттоя мыджехемгитаиь ныго коб саглто а всгемая задта ныга коб темы а пхаякты чке отто чвмскмемого синя типхескакиглте счоскгамеш чиь неглуи ето вдтомого мхетсбере киь неглуи ето гшнаго ире ето гшнаго ире до ире соьере а до ире гшнемл б тий пегтеи енгожотаи аь ныге ий пескеятте хожескте нгадескл ире ий мсиржо ныго пхаякто мси чихкы ире эохобкихо бекехый ето вдтомого неглуи а неглу

Результат, как и на основе статистики биграмм, получился высокоточным.

Для произвольного текста, результат во многом будет зависеть от длины сообщения, начального приближения (технически, можно застрять в локальном максимуме) а также от энтропии конкретного языка.

## Часть 6.

Бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

Конечно, подобные методологии вряд ли могут быть всерьез применены в криптографии (кажется, даже для шифра Вижинера уже непросто будет построить подобный метод, не говоря о современных алгоритмах). В тоже время, основной особенностью задачи было использование МСМС-сэмплирования для нахождения максимума функции (фактически, рассмотренное блуждание Метрополиса-Гастингса делает с большей вероятностью шаг в сторону увеличения плотности). Таким образом, рассмотренный подход может использоваться, например, как итеративный алгоритм поиска максимума функции (например, для оценки статистических характеристик распределения). 